# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
from sklearn.decomposition import PCA, TruncatedSVD
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from collections import Counter

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

/opt/anaconda3/envs/yt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Linguistic Modules

In [2]:
def preprocess_text(text):
    # Define interrogative words to KEEP
    interrogatives = {"what", "why", "how", "who", "where", "when", "which", "whom", "whose", "no", "not",
                    "very" ,"too" ,"too" ,"just", "if", "but", "however", "without", "like"}
    custom_stopwords = set(nlp.Defaults.stop_words)
    custom_stopwords -= interrogatives

    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
# Process tokens: lemmatize, filter stopwords/punct/numbers, keep interrogatives
    tokens = [
        token.lemma_ 
        for token in doc 
        if (
            (not token.is_stop or token.text in interrogatives) and  # Keep interrogatives
            not token.is_punct and token.is_alpha                                  # Remove punctuation
            # (token.is_alpha or token.like_num)                       # Keep words/numbers
        )
    ]

    return ' '.join(tokens)

In [3]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'number_present': any(token.pos_ == 'NUM' for token in doc)
    }
    return features

In [4]:
def get_sentence_vector(text):
    words = text.strip().split()
    vectors = [model.get_word_vector(w) for w in words]
    if not vectors:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

# Pre-Processing

### Import Data

In [ ]:
label_mapper = {
    'BT1' : 'knowledge',
    'BT2' : 'comprehension',
    'BT3' : 'application',
    'BT4' : 'analysis',
    'BT5' : 'synthesis',
    'BT6' : 'evaluation'
}

# Load dataset
df = pd.DataFrame()
for i in range(1,5):
    q_df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/dataset' + str(i) + '.csv')
    df = pd.concat([df , q_df])

# Apply preprocessing
mask = df['label'].isin(label_mapper.keys())
df['label'] = df['label'].mask(mask, df['label'].map(label_mapper))

df['label'] = df['label'].str.lower()

df['processed_question'] = df['question'].apply(preprocess_text)

df['processed_question'] = [''.join(text) for text in df['processed_question']]

: 

## Tokenize

BERT model is used to retain context

In [ ]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['question'].tolist(), return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

bert_df = pd.DataFrame(sentence_embedding.numpy())

bert_df.to_csv('bert_sentence_embeddings.csv', index=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Feature Extraction

### Adding Linguistic Features

Execute if linguistic features like count for verb,noun,etc needed

In [ ]:
# Adding linguistic features
ling_features = df['question'].apply(extract_linguistic_features).apply(pd.Series)

# Concatenate DataFrames
token_df = pd.concat([ling_features, bert_df], axis=1)

token_df.columns = token_df.columns.map(str)

### Label Balancing

In [22]:
x , y = token_df , df['label']
x_resampled , y_resampled = SMOTETomek().fit_resample(x , y)

# Modelling

## Basic Machine Learning

In [23]:
x_train , x_test , y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=34 , stratify= y_resampled)

### Random Forest

In [24]:
params = {
    'n_estimators' : [2 , 5 , 10 , 100, 250 , 500],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 100 , 250 , 500]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5, n_jobs= -1)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,mean_test_score,params
4,0.815142,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 250}"
5,0.814424,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 500}"
3,0.814305,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}"
2,0.789733,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 10}"
1,0.770601,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 5}"
0,0.676436,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 2}"
9,0.596204,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 100}"
8,0.484939,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}"
7,0.392208,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 5}"
6,0.297926,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 2}"


In [25]:
classifier = RandomForestClassifier(n_estimators= 250 , max_depth= None , criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.94      0.89      0.91       563
  application       0.92      0.91      0.91       563
comprehension       0.87      0.87      0.87       554
   evaluation       0.80      0.75      0.78       559
    knowledge       0.78      0.86      0.82       548
    synthesis       0.77      0.79      0.78       559

     accuracy                           0.85      3346
    macro avg       0.85      0.85      0.85      3346
 weighted avg       0.85      0.85      0.85      3346



In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
input =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
output = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
question_embedding = outputs.last_hidden_state[:, 0, :]

bert_q = pd.DataFrame(sentence_embedding.numpy())

# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, bert_q], axis=1)

print(classifier.predict(token_q))

### Decision Tree

In [27]:
params = {
    'min_samples_split' : [2 , 5 , 10 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50 , 100 , 250 , 500]
    # 'min_samples_leaf' : [1 ,2 , 5 , 10 , 20 , 50 , 100]
}
param_search = GridSearchCV(estimator= DecisionTreeClassifier(), param_grid= params, cv=5, n_jobs= -1)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,mean_test_score,params
0,0.751050,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}"
1,0.747344,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 5}"
2,0.738196,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 10}"
3,0.711950,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 50}"
4,0.684508,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 100}"
5,0.340731,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2}"
7,0.340731,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 10}"
6,0.340551,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5}"
8,0.340551,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 50}"
9,0.340372,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 100}"


In [28]:
dt = DecisionTreeClassifier(criterion= 'gini', max_depth= None, min_samples_split = 2)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.85      0.84      0.85       563
  application       0.84      0.86      0.85       563
comprehension       0.83      0.81      0.82       554
   evaluation       0.72      0.75      0.73       559
    knowledge       0.74      0.75      0.75       548
    synthesis       0.74      0.70      0.72       559

     accuracy                           0.78      3346
    macro avg       0.78      0.78      0.78      3346
 weighted avg       0.78      0.78      0.78      3346



In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
input =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
output = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
question_embedding = outputs.last_hidden_state[:, 0, :]

bert_q = pd.DataFrame(sentence_embedding.numpy())

# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, bert_q], axis=1)

print(dt.predict(token_q))

['analysis']


### XGBoost

In [ ]:
y_mapper = {
    'knowledge' : 0,
    'comprehension' : 1,
    'application' : 2,
    'analysis' : 3,
    'synthesis' : 4,
    'evaluation' : 5
}

xgb_y = y_resampled.map(y_mapper)
xgb_y_train = y_train.map(y_mapper)
xgb_y_test = y_test.map(y_mapper)


In [ ]:
params = {
    'objective' : ['multi:softprob'],
    'max_depth' : [None , 10 , 50 , 100 , 250 , 500],
    'n_estimators': [2 , 10 , 50 , 100, 200],
    'learning_rate': [0.01, 0.1 , 0.5, 1],
    'subsample': [0.8, 0.9]
}
param_search = GridSearchCV(estimator= XGBClassifier(), param_grid= params, cv=5, n_jobs= -1)

param_search.fit(x_resampled , xgb_y)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
xgb = XGBClassifier(criterion= 'gini', max_depth= None, min_samples_split = 2)
xgb.fit(x_train, xgb_y_train)
y_pred = xgb.predict(x_test)
print(classification_report(xgb_y_test , y_pred))

In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
input =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
output = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
question_embedding = outputs.last_hidden_state[:, 0, :]

bert_q = pd.DataFrame(sentence_embedding.numpy())

# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, bert_q], axis=1)

print(xgb.predict(token_q))

### SVM

In [ ]:
params = {
    'C': [0.001, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear','poly', 'sigmoid'],
    'degree': [1, 2, 4],
}
param_search = GridSearchCV(estimator= XGBClassifier(), param_grid= params, cv=5, n_jobs= -1)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 